In [ ]:
# Dependencies:
import numpy as np
import pandas as pd
import scipy.stats as stat
from matplotlib import pyplot as plt
from config import api_key
from pprint import pprint
import requests
import math


In [155]:
# Data 
# global netflix data from netflix.com top 10 weekly shows
global_netflix_df = pd.read_csv("Resources/all-weeks-global.csv")
global_netflix_df.head()
global_netflix_df.dtypes

# Netflix Title data from kaggle.com- has title, info, imdb ratings, etc.
titles_netflix_df = pd.read_csv("Resources/titles.csv")
titles_netflix_df.head()
titles_netflix_df.dtypes

# Merge 
global_titles_df = pd.merge(global_netflix_df, titles_netflix_df, how= "left", left_on="show_title", right_on="title")
global_titles_df.drop(["title","tmdb_popularity","tmdb_score"], axis=1, inplace=True)
global_titles_df

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,cumulative_weeks_in_top_10,id,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,2022-10-30,Films (English),1,The Good Nurse,NaN,68310000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-30,Films (English),2,The School for Good and Evil,NaN,41950000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-30,Films (English),3,The Stranger,NaN,9170000,2,ts224901,SHOW,A web of secrets sends family man Adam Price o...,2020.0,TV-MA,45.0,"['thriller', 'crime', 'drama']",['GB'],1.0,tt9698480,7.3,48298.0
3,2022-10-30,Films (English),4,Wild Is the Wind,NaN,8890000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-30,Films (English),5,The Curse of Bridge Hollow,NaN,8170000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,2021-07-04,TV (Non-English),6,Elite,Elite: Season 1,10530000,1,ts78305,SHOW,When three working class kids enroll in the mo...,2018.0,TV-MA,48.0,"['thriller', 'drama', 'crime']",['ES'],5.0,tt7134908,7.4,75337.0
2816,2021-07-04,TV (Non-English),7,Elite,Elite: Season 3,10200000,1,ts78305,SHOW,When three working class kids enroll in the mo...,2018.0,TV-MA,48.0,"['thriller', 'drama', 'crime']",['ES'],5.0,tt7134908,7.4,75337.0
2817,2021-07-04,TV (Non-English),8,Elite,Elite: Season 2,10140000,1,ts78305,SHOW,When three working class kids enroll in the mo...,2018.0,TV-MA,48.0,"['thriller', 'drama', 'crime']",['ES'],5.0,tt7134908,7.4,75337.0
2818,2021-07-04,TV (Non-English),9,Katla,Katla: Season 1,9190000,1,ts237972,SHOW,"A year after a subglacial volcano erupts, myst...",2021.0,TV-MA,46.0,"['drama', 'thriller', 'scifi']",['IS'],1.0,tt11102190,7.0,13472.0


In [ ]:
global_titles_df.to_csv("output_data/global_titles.csv")

In [ ]:
url = "http://www.omdbapi.com/?t="
api_key = "&apikey=b9b46bfd"

In [94]:
# Changing all NAN values to NA in global_titles_df for easy referencing 
df = global_titles_df.fillna("NA")
df
# filtering data where imdb_score is NA and respective show title
df1 = df.loc[df["imdb_score"]=="NA","show_title"]
# Created the movie list with unique movie/show names
movies_list = df1.unique().tolist()
# Total number of unique movies/shows
len(movies_list)

418

In [148]:
response = requests.get(url+"narco-saints"+api_key).json()
response

{'Title': 'Narco Saints',
 'Year': '2019–',
 'Rated': 'N/A',
 'Released': '05 Feb 2019',
 'Runtime': 'N/A',
 'Genre': 'Crime',
 'Director': 'N/A',
 'Writer': 'N/A',
 'Actors': 'N/A',
 'Plot': 'The life-threatening journey of a civilian businessman who has no choice but to cooperate with the secret operation of the National Intelligence Service.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BYjRmOWJjYTQtNGRjYi00ZTJkLTliYTAtN2U2NGQ3ZmYyNjM4XkEyXkFqcGdeQXVyNDQyNDAzMTk@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}],
 'Metascore': 'N/A',
 'imdbRating': '7.2',
 'imdbVotes': '200',
 'imdbID': 'tt9631156',
 'Type': 'series',
 'totalSeasons': '1',
 'Response': 'True'}

In [93]:
# List that has unique movie/show names
movies_list
# Empty list to store fetched data from API
data = []
# API Key
api_key= "&apikey=b9b46bfd"
# looping through the movie list and fetching data for each movie
for movie in movies_list:
    try:
        response = requests.get(url+movie+api_key).json()
        # Appending data to the list
        data.append({"show_title":response["Title"],
                    "id" :response["imdbID"],"type":response["Type"],"description":response["Plot"],
                    "release_year":response["Year"],"age_certification":response["Rated"],"runtime":response["Runtime"],
                    "genres":response["Genre"],"production_countries":response["Country"],"imdb_id":response["imdbID"],
                    "imdb_score":response["imdbRating"],"imdb_votes":response["imdbVotes"]})
    except KeyError:
        pass
        
# Created the dataframe with all data fetched from api
#df3 = pd.DataFrame(data)
#df3


,show_title,id,type,description,release_year,age_certification,runtime,genres,production_countries,imdb_id,imdb_score,imdb_votes
0,The Good Nurse,tt4273800,movie,An infamous caregiver is implicated in the dea...,2022,R,121 min,"Biography, Crime, Drama",United States,tt4273800,7.1,204
1,The School for Good and Evil,tt2935622,movie,Best friends Sophie and Agatha find themselves...,2022,PG-13,147 min,"Action, Comedy, Drama",United States,tt2935622,5.8,"18,952"
2,Wild Is the Wind,tt0051193,movie,An immigrant Nevada rancher brings a woman fro...,1957,Not Rated,114 min,Drama,United States,tt0051193,6.7,"1,027"
3,The Curse of Bridge Hollow,tt15289240,movie,"A teenage girl, who accidentally releases an a...",2022,TV-14,89 min,"Adventure, Comedy, Family",United States,tt15289240,N/A,"4,085"
4,Luckiest Girl Alive,tt4595186,movie,"A woman in New York, who seems to have things ...",2022,R,113 min,"Drama, Mystery, Thriller",United States,tt4595186,6.4,"13,563"
...,...,...,...,...,...,...,...,...,...,...,...,...
367,How to Train Your Dragon: The Hidden World,tt2386490,movie,When Hiccup discovers Toothless isn't the only...,2019,PG,104 min,"Animation, Action, Adventure","United States, Japan",tt2386490,7.4,"128,891"
368,The Huntsman: Winter's War,tt2381991,movie,"Eric and fellow warrior Sara, raised as member...",2016,PG-13,114 min,"Action, Adventure, Drama","United States, China",tt2381991,6.1,"110,798"
369,O Candidato Honesto,tt4083076,movie,John is a corrupt politician wanting to be the...,2014,N/A,110 min,Comedy,Brazil,tt4083076,5.4,"1,094"
370,Fatale,tt8829830,movie,"After a one-night stand, a successful married ...",2020,R,102 min,Thriller,United States,tt8829830,5.4,"10,407"


In [180]:
df3 = pd.DataFrame(data)
df3
# Exporting data fetched from the API to csv file
df3.to_csv("output_data/api_data.csv")

In [187]:
# merged the data fetched(df3) and dataframe we have(global_titles_df)
#changing all show titles into lower case 
global_titles_df["Show_title"]=global_titles_df['show_title'].str.lower()
df3["Show_title"]=df3['show_title'].str.lower()

merge_df = pd.merge(global_titles_df, df3, how= "left",on="Show_title")
# combining the data and storing it in separate column.(eg:combined type_x,type_y into one column type) and data cleaning
combined_df = merge_df.assign(type=merge_df.type_x.fillna(merge_df.type_y))
combined_df = combined_df.assign(description=merge_df.description_x.fillna(merge_df.description_y))
combined_df = combined_df.assign(release_year=merge_df.release_year_x.fillna(merge_df.release_year_y))
combined_df = combined_df.assign(age_certification=merge_df.age_certification_x.fillna(merge_df.age_certification_y))
combined_df = combined_df.assign(runtime=merge_df.runtime_x.fillna(merge_df.runtime_y))
combined_df = combined_df.assign(genres=merge_df.genres_x.fillna(merge_df.genres_y))
combined_df = combined_df.assign(production_countries=merge_df.production_countries_x.fillna(merge_df.production_countries_y))
combined_df = combined_df.assign(imdb_id=merge_df.imdb_id_x.fillna(merge_df.imdb_id_y))
combined_df = combined_df.assign(imdb_score=merge_df.imdb_score_x.fillna(merge_df.imdb_score_y))
combined_df = combined_df.assign(imdb_votes=merge_df.imdb_votes_x.fillna(merge_df.imdb_votes_y))
#dropping used columns
combined_df.drop(["show_title_x","show_title_y","id_x","id_y","type_x","type_y","description_x","description_y","release_year_x",
                "release_year_y","age_certification_x","age_certification_y","runtime_x","runtime_y","genres_x","genres_y",
                "production_countries_x","production_countries_y","imdb_id_x","imdb_id_y","imdb_score_x","imdb_score_y",
                "imdb_votes_x","imdb_votes_y","season_title","seasons"], axis=1, inplace=True)
#dropped all rows that has NAN values 
combined_df = combined_df.dropna()
# Dropped all rows that has imdb_score equal to N/A
cleaned_df = combined_df.loc[combined_df["imdb_score"]!="N/A"]
cleaned_df = combined_df.loc[combined_df["runtime"]!="N/A"]
cleaned_df = combined_df.loc[combined_df["age_certification"]!="N/A"]
#saving the dataframe into csv file
cleaned_df.to_csv("output_data/cleaned_netflix_data.csv")
print(cleaned_df.count())
cleaned_df

week                          2069
category                      2069
weekly_rank                   2069
weekly_hours_viewed           2069
cumulative_weeks_in_top_10    2069
Show_title                    2069
type                          2069
description                   2069
release_year                  2069
age_certification             2069
runtime                       2069
genres                        2069
production_countries          2069
imdb_id                       2069
imdb_score                    2069
imdb_votes                    2069
dtype: int64


,week,category,weekly_rank,weekly_hours_viewed,cumulative_weeks_in_top_10,Show_title,type,description,release_year,age_certification,runtime,genres,production_countries,imdb_id,imdb_score,imdb_votes
0,2022-10-30,Films (English),1,68310000,1,the good nurse,movie,An infamous caregiver is implicated in the dea...,2022,R,121 min,"Biography, Crime, Drama",United States,tt4273800,7.1,204
1,2022-10-30,Films (English),2,41950000,2,the school for good and evil,movie,Best friends Sophie and Agatha find themselves...,2022,PG-13,147 min,"Action, Comedy, Drama",United States,tt2935622,5.8,"18,952"
2,2022-10-30,Films (English),3,9170000,2,the stranger,SHOW,A web of secrets sends family man Adam Price o...,2020,TV-MA,45,"['thriller', 'crime', 'drama']",['GB'],tt9698480,7.3,48298
3,2022-10-30,Films (English),4,8890000,1,wild is the wind,movie,An immigrant Nevada rancher brings a woman fro...,1957,Not Rated,114 min,Drama,United States,tt0051193,6.7,"1,027"
4,2022-10-30,Films (English),5,8170000,3,the curse of bridge hollow,movie,"A teenage girl, who accidentally releases an a...",2022,TV-14,89 min,"Adventure, Comedy, Family",United States,tt15289240,N/A,"4,085"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,2021-07-04,TV (Non-English),6,10530000,1,elite,SHOW,When three working class kids enroll in the mo...,2018,TV-MA,48,"['thriller', 'drama', 'crime']",['ES'],tt7134908,7.4,75337
2816,2021-07-04,TV (Non-English),7,10200000,1,elite,SHOW,When three working class kids enroll in the mo...,2018,TV-MA,48,"['thriller', 'drama', 'crime']",['ES'],tt7134908,7.4,75337
2817,2021-07-04,TV (Non-English),8,10140000,1,elite,SHOW,When three working class kids enroll in the mo...,2018,TV-MA,48,"['thriller', 'drama', 'crime']",['ES'],tt7134908,7.4,75337
2818,2021-07-04,TV (Non-English),9,9190000,1,katla,SHOW,"A year after a subglacial volcano erupts, myst...",2021,TV-MA,46,"['drama', 'thriller', 'scifi']",['IS'],tt11102190,7,13472
